In [ ]:
import nltk  # Imports the nltk library for natural language processing
from newspaper import Article  # Imports the Article class from the newspaper module
from googletrans import Translator  # Imports the Translator class from the googletrans module
import tkinter as tk  
from tkinter import *  # Imports all components from the tkinter module

In [ ]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from googletrans import Translator
from newspaper import Article
from PIL import Image, ImageTk
import requests
from io import BytesIO
import nltk

# Create the main window
top = tk.Tk()
top.geometry('800x800')
top.title('News Article Summary')
top.configure(bg='#2c3e50')

# Define custom fonts and styles
title_font = ('Helvetica', 24, 'bold')
label_font = ('Arial', 14, 'bold')
content_font = ('Verdana', 12)
button_font = ('Verdana', 12, 'bold')

# Styling for buttons and other elements
style = ttk.Style()
style.configure('TButton', font=button_font, background='#1abc9c', foreground='#ffffff')
style.map("TButton", background=[('active', '#16a085')])

# Variables used for the functions
article = ''
art_title = ''
article_sum = ''
art_content = ''

# Canvas and frame for scrollable content
my_canvas = Canvas(top, borderwidth=0, background="#ecf0f1", highlightthickness=0)
frame = Frame(my_canvas, background='#ecf0f1', padx=20, pady=20)
vsb = Scrollbar(top, orient="vertical", command=my_canvas.yview)
my_canvas.configure(yscrollcommand=vsb.set)

my_canvas.pack(side='left', fill='both', expand=True, padx=20, pady=20)
vsb.pack(side='right', fill='y', padx=10, pady=10)
my_canvas.create_window((0, 0), window=frame, anchor='nw')
frame.bind("<Configure>", lambda event: my_canvas.configure(scrollregion=my_canvas.bbox("all")))

# Translating and Summary Function
def translate_news():
    global article
    global art_title
    global article_sum
    global art_content

    url = news_url.get()
    article = Article(url)
    article.download()
    article.parse()
    nltk.download('punkt')
    article.nlp()

    # Translating Article
    translator = Translator()
    art_title = translator.translate(article.title, dest=btn2.get())
    art_content = translator.translate(article.text, dest=btn2.get())
    article_sum = translator.translate(article.summary, dest=btn2.get())

    # Clear previous content
    for widget in frame.winfo_children():
        widget.destroy()

    # Displaying Title
    title_label = Label(frame, text="Article Title", font=label_font, background='#ecf0f1', fg='#34495e')
    title_label.pack(pady=3)

    article_title = Text(frame, height=2, width=65, wrap='word', font=content_font, borderwidth=2)
    article_title.insert('end', art_title.text)
    article_title.config(state="disabled", background='#f5f6fa')
    article_title.pack(pady=1)

    # Displaying Summary
    summary_label = Label(frame, text="Article Summary:", font=label_font, background='#ecf0f1', fg='#34495e')
    summary_label.pack(pady=3)

    text_box = Text(frame, height=8, width=65, wrap='word', font=content_font, borderwidth=2)
    text_box.insert('end', article_sum.text)
    text_box.config(state="disabled", background='#f5f6fa')
    text_box.pack(fill=BOTH, expand=True, pady=5)

    # Displaying Publish Date
    publish_label = Label(frame, text="Article Publish Date:", font=label_font, background='#ecf0f1', fg='#34495e')
    publish_label.pack(pady=3)
    
    article_publish = Text(frame, height=1, width=65, font=content_font, borderwidth=2)
    article_publish.insert('end', article.publish_date if article.publish_date else "Unknown")
    article_publish.config(state="disabled", background='#f5f6fa')
    article_publish.pack(pady=5)

    # Displaying Article Image (if available)
    if article.top_image:
        link_label = Label(frame, text="Article Image:", font=label_font, background='#ecf0f1', fg='#34495e')
        link_label.pack(pady=3)

        response = requests.get(article.top_image)
        img_data = response.content
        img = Image.open(BytesIO(img_data))
        img = img.resize((400, 300), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)

        img_label = Label(frame, image=img, background='#ecf0f1')
        img_label.image = img  # Keep a reference of the image
        img_label.pack(pady=5)
    else:
        link_label = Label(frame, text="No image available", font=label_font, background='#ecf0f1', fg='#e74c3c')
        link_label.pack(pady=3)

# Downloading article function
def download_article():
    with open("News_summary_File.txt", "w+", encoding="utf-8") as file1:
        file1.write("Title:\n")
        file1.write(art_title.text + "\n\n")
        file1.write("Article Text:\n")
        file1.write(art_content.text + "\n\n")
        file1.write("Article Summary:\n")
        file1.write(article_sum.text + "\n\n")
        file1.write("Article Publish Date:\n")
        file1.write(str(article.publish_date) + "\n\n")
        file1.write("Article Image Link:\n")
        file1.write(article.top_image)

# Adding a title to the window
header_label = Label(top, text="News Article Translator and Summarizer", font=title_font, background='#2c3e50', fg='#ffffff')
header_label.pack(pady=20)

# URL Entry field
Label(frame, text="Enter News Article URL", font=label_font, background='#ecf0f1', fg='#34495e').pack(pady=5)
news_url = Entry(frame, width=65, borderwidth=2, font=('verdana', 11))
news_url.configure(highlightbackground="red", highlightcolor="red")
news_url.pack(pady=5)

# Language selection
my_label = Label(frame, text="Select Language to Translate", font=label_font, background='#ecf0f1', fg='#34495e')
my_label.pack(pady=5)

# Radio buttons for language selection
font_radio = ('calibri', 13)
btn2 = StringVar(value="en")
Radiobutton(frame, text="English", value="en", variable=btn2, font=font_radio, background='#ecf0f1').pack(pady=5)
Radiobutton(frame, text="Hindi", value="hi", variable=btn2, font=font_radio, background='#ecf0f1').pack(pady=5)
Radiobutton(frame, text="Marathi", value="mr", variable=btn2, font=font_radio, background='#ecf0f1').pack(pady=5)

# Translation and Summarization button
translate_button = ttk.Button(frame, text="Translate and Summarize", command=translate_news)
translate_button.pack(pady=15)


top.mainloop()


